In [36]:
import numpy as np 
import pandas as pd
import os

from scipy import stats
from scipy.stats import norm

from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

import pickle

import warnings
warnings.filterwarnings('ignore')

# custom general helper functions for this project
import custom_utils as cu
import importlib

In [37]:
# reload imports as needed
importlib.reload(cu);

In [7]:
# Customize matplotlib default settings
matplotlib.rcParams.update({'font.size': 16})
plt.rcParams["figure.figsize"] = (20,10)

In [8]:
# plotly
import plotly.plotly as py
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go

## 1. Get stock data

In [9]:
!pip install alpha_vantage

  Stored in directory: /Users/arina/Library/Caches/pip/wheels/21/9c/02/b7cfae63f7119f4089f229ba1a09ce90fabbaf96199c2890be
Successfully built alpha-vantage


In [10]:
from alpha_vantage.timeseries import TimeSeries

In [41]:
# I've saved my Alpha Vantage API key in a separate file, so that I can just read it in here with my helper function
cu.list_api_keys()

['alpha_vantage']

In [42]:
av_api_key = cu.get_api_key("alpha_vantage")

In [44]:
# ts = TimeSeries(key='YOUR_API_KEY', output_format='pandas')
ts = TimeSeries(key=av_api_key, output_format='pandas')

In [45]:
# Get the daily stock data and the call's metadata
stock_data, stock_meta_data = ts.get_daily('GOOGL', outputsize='full')

In [46]:
stock_meta_data

{'1. Information': 'Daily Prices (open, high, low, close) and Volumes',
 '2. Symbol': 'GOOGL',
 '3. Last Refreshed': '2019-04-17',
 '4. Output Size': 'Full size',
 '5. Time Zone': 'US/Eastern'}

In [47]:
stock_data.head()

,1. open,2. high,3. low,4. close,5. volume
date,,,,,
2004-08-19,100.01,104.06,95.96,100.335,44659000.0
2004-08-20,101.01,109.08,100.50,108.310,22834300.0
2004-08-23,110.76,113.48,109.05,109.400,18256100.0
2004-08-24,111.24,111.60,103.57,104.870,15247300.0
2004-08-25,104.76,108.00,103.88,106.000,9188600.0


In [48]:
# rename columns so that they only have varname-acceptible characters in them
stock_data.columns = ["open", "high", "low", "close", "volume"]

In [49]:
# pickle the data 
myoutfile = "google_stock_data.pkl"
with open(myoutfile, 'wb') as picklefile:
     pickle.dump(stock_data, picklefile)

In [50]:
# unpickle
with open("google_stock_data.pkl", 'rb') as picklefile: 
    stock_data = pickle.load(picklefile)

stock_data.head()

,open,high,low,close,volume
date,,,,,
2004-08-19,100.01,104.06,95.96,100.335,44659000.0
2004-08-20,101.01,109.08,100.50,108.310,22834300.0
2004-08-23,110.76,113.48,109.05,109.400,18256100.0
2004-08-24,111.24,111.60,103.57,104.870,15247300.0
2004-08-25,104.76,108.00,103.88,106.000,9188600.0


In [51]:
stock_data.tail()

,open,high,low,close,volume
date,,,,,
2019-04-11,1208.90,1212.74,1204.54,1209.59,849947.0
2019-04-12,1215.62,1223.05,1213.29,1222.73,1215610.0
2019-04-15,1224.09,1229.30,1214.56,1226.53,1189974.0
2019-04-16,1230.00,1235.98,1225.04,1231.91,1131099.0
2019-04-17,1237.00,1245.10,1232.90,1240.14,1471225.0


## 2. Data exploration

In [52]:
stock_data.head()

,open,high,low,close,volume
date,,,,,
2004-08-19,100.01,104.06,95.96,100.335,44659000.0
2004-08-20,101.01,109.08,100.50,108.310,22834300.0
2004-08-23,110.76,113.48,109.05,109.400,18256100.0
2004-08-24,111.24,111.60,103.57,104.870,15247300.0
2004-08-25,104.76,108.00,103.88,106.000,9188600.0


In [53]:
stock_data.describe()

,open,high,low,close,volume
count,3691.000000,3691.000000,3691.000000,3691.000000,3.691000e+03
mean,635.167468,641.086776,628.712554,634.970676,7.387645e+06
std,259.306949,260.807566,257.595840,259.319778,8.095545e+06
min,99.090000,101.740000,95.960000,100.010000,3.845900e+04
25%,469.795000,474.105000,464.680000,469.975000,2.091950e+06
50%,580.000000,585.000000,574.190000,580.110000,4.645500e+06
75%,794.975000,799.955000,788.355000,793.485000,9.610600e+06
max,1289.120000,1291.440000,1263.000000,1285.500000,8.215110e+07


In [54]:
print("Date range:", stock_data.index.min(), "to", stock_data.index.max())

Date range: 2004-08-19 to 2019-04-17


In [55]:
# datarow with the max closing price
stock_data[stock_data.close == stock_data.close.max()]

,open,high,low,close,volume
date,,,,,
2018-07-26,1267.18,1287.4,1263.0,1285.5,2734302.0


In [56]:
# datarow with the max volume
stock_data[stock_data.volume == stock_data.volume.max()]

,open,high,low,close,volume
date,,,,,
2006-01-20,437.5,440.03,394.74,399.4601,82151100.0


In [57]:
trace = go.Scatter(
                    x = stock_data.index,
                    y = stock_data.close,
                    mode = "lines",
                    name = "daily close prices",
                    marker = dict(color = 'rgba(16, 112, 2, 0.8)'),
                    text= stock_data.index)

layout = dict(title = 'GOOGL Stock Daily Close Prices',
              xaxis= dict(title= 'Date', zeroline= False),
              yaxis= dict(title= 'Close price')
             )
fig = dict(data = [trace], layout = layout)
iplot(fig)

In [ ]:
# from keras.models import Sequential
# from keras.layers import Dense
# from keras.layers import Dropout